# MLT-03 ANN Concepts

- Authored by: *Jay Parmar*
- Last modified on: *27th August 2023*

## In this notebook, we will go through following topics:

- Feature Engineering
- Neural Network Creation
- Strategy Backtesting

The focus of this lab would be to understand the implementation of neural networks using the scikit-learn library. To do so, we will start with importing necessary libraries.

## Part 1 - Training a neural network

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Apply the default seaborn theme, scaling, and color palette
sns.set()
# One can use different colour palettes
# palettes = ["deep", "muted", "pastel", "bright", "dark", "colorblind"]
# sns.set(palette="deep")

# import warnings
# warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
# Loading the data from the local file
df = pd.read_csv('EURUSD_data.csv', index_col=0, parse_dates=True)

In [ ]:
# Copying the original dataframe. Will work on the new dataframe.
data = df.copy()
# Checking the shape
print('Number of observations:', data.shape[0])
print('Number of variables:', data.shape[1])

In [ ]:
data.head()

### Feature Engineering

In [ ]:
# Creating features
features_list = []

# SD based features
for i in range(5, 20, 5):
    col_name = 'std_' + str(i)
    data[col_name] = data['Close'].rolling(window=i).std()
    features_list.append(col_name)
    
# MA based features
for i in range(10, 30, 5):
    col_name = 'ma_' + str(i)
    data[col_name] = data['Close'].rolling(window=i).mean()
    features_list.append(col_name)
    
# Daily pct change based features
for i in range(3, 12, 3):
    col_name = 'pct_' + str(i)
    data[col_name] = data['Close'].pct_change().rolling(i).sum()
    features_list.append(col_name)
    
# Intraday movement
col_name = 'co'
data[col_name] = data['Close'] - data['Open']
features_list.append(col_name)

In [ ]:
features_list

In addition, We'll use popular technical indicators to build features. They are as follows:

- [Bollinger Bands](https://en.wikipedia.org/wiki/Bollinger_Bands)
- [Moving Average Convergence/Divergence (MACD)](https://en.wikipedia.org/wiki/MACD)
- [Parabolic Stop And Reverse (SAR)](https://en.wikipedia.org/wiki/Parabolic_SAR)


The discussion about what these technical indicators and how they are built, is out of scope of this session. We'll use `TA-LIB` library to build these indicators.

In [ ]:
# Use the following command on the terminal window on Anaconda to install ta-lib if it is not installed
# conda install -c conda-forge ta-lib
import talib as ta

In [ ]:
data['upper_band'], data['middle_band'], data['lower_band'] = ta.BBANDS(data['Close'].values)
data['macd'], data['macdsignal'], data['macdhist'] = ta.MACD(data['Close'].values)
data['sar'] = ta.SAR(data['High'].values, data['Low'].values)
features_list +=['upper_band','middle_band','lower_band','macd','sar']

ML algorithms don't work with `NaN` values. However, while creating the above features, we would have many `NaN` values we need to drop from our dataset.

In [ ]:
features_list

In [ ]:
data[features_list].head()

There are null values in many columns. Let's drop them.

In [ ]:
data.dropna(inplace=True)

In [ ]:
data[features_list].head()

As OHLC data is high correlated, we won't be using them as features. Instead, we would use only technical indicators and quantitative features for this exercise. Below we define feature matrix `X`, create the target variable and assign it to the target vector `y`.

In [ ]:
import numpy as np

In [ ]:
X = data[features_list]
data['target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)
y = data['target']

We will use `train_test_split()` function from the `sklearn.model_selection` package to split our dataset. We will use 20% of our dataset as a test dataset.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Before we can train our neural network, we need to make sure that our data is scaled, that is, it ranges between 0 and 1. We will use `StandardScaler` from the `sklearn.preprocessing` package. We need to train the scaler object on training data only and then apply on training and testing set both.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
#sns.pairplot(X_train_scaled_df[features_list]);

In [ ]:
X_train_scaled_df.describe().round(2)

We have everything ready now. Now is the time to create our first neural network. We'll use the `MLPClassifier` from the `sklearn.neural_network` package. Here, MLP stands for Multi Layer Perceptron. A simple neural network is shown below:

![Neural Network](https://www.learnopencv.com/wp-content/uploads/2017/10/mlp-diagram-600x400.jpg)

We can see that a neural network consists of

- Input layer,
- Hidden layer, and 
- Output layer

Hence, we need to define these layers for the model. In our case, the feature matrix `X` becomes input to the input layer. Then we have hidden layer/s and finally the output layer. In the above diagram, we have only one hidden layer.

    Note: In `sklearn` library, we need not specify the size of the input and output layer. It will be fixed by the library itself when we train it. Hence, we are only to define hidden layer sizes.

Below we define the model:

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
# Define model
# model = MLPClassifier(hidden_layer_sizes=(5), verbose=True, random_state=10)
model = MLPClassifier(hidden_layer_sizes=(5), max_iter=300, activation = 'tanh', solver='adam', random_state=1, shuffle=False)

# Train model
model.fit(X_train_scaled, y_train)

Congratulations, we have successfully trained our first neural network. Now let's check its properties

In [ ]:
# Check number of layers in the model
model.n_layers_

In [ ]:
model.get_params()

In [ ]:
# Check weights
print('Weights between input layer and the hidden layer:')
print(model.coefs_[0])
print('Biases between input layer and the hidden layer:')
print(model.intercepts_[0])

In [ ]:
print('Weights between hidden layer and the output layer:')
print(model.coefs_[1])
print('Biases between hidden layer and the output layer:')
print(model.intercepts_[1])

In [ ]:
# Check model accuracy on training data
print('Model accuracy on training data:', model.score(X_train_scaled, y_train))

In [ ]:
# Check model accuracy on testing data
print('Model accuracy on testing data:', model.score(X_test_scaled, y_test))

In [ ]:
# Predict data
y_pred = model.predict(X_test_scaled)

In [ ]:
y_pred

As training and testing accuracy are very similar, we can consider that model might not have overfitted, and it may generalize well. However, it is difficult to claim until we evaluate the model properly.

Also, the model that we have created is a very simple one; we have used most of the default parameters for building the model. And they might not be the best one. 

In [ ]:
# Calculate Precision and Recall
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
print("Precision:", precision)
print("Recall:", recall)

## Part 2 - Backtesting our predictions

So far we've covered
* Read Data
* Create Features
* Scale data
* Use already trained model to make predictions
* Trade on those prediction, and calculate the strategy returns

In [ ]:
def backtest(df, model):
    # Copy data
    data = df.copy()
    
    # Create returns
    data['returns'] = np.log(data['Close'] / data['Close'].shift(1))
    # Creating features
    features_list = []

    # SD based features
    for i in range(5, 20, 5):
        col_name = 'std_' + str(i)
        data[col_name] = data['Close'].rolling(window=i).std()
        features_list.append(col_name)

    # MA based features
    for i in range(10, 30, 5):
        col_name = 'ma_' + str(i)
        data[col_name] = data['Close'].rolling(window=i).mean()
        features_list.append(col_name)

    # Daily pct change based features
    for i in range(3, 12, 3):
        col_name = 'pct_' + str(i)
        data[col_name] = data['Close'].pct_change().rolling(i).sum()
        features_list.append(col_name)

    # Intraday movement
    col_name = 'co'
    data[col_name] = data['Close'] - data['Open']
    features_list.append(col_name)
    # Create features
    data['upper_band'], data['middle_band'], data['lower_band'] = ta.BBANDS(data['Close'].values)
    data['macd'], data['macdsignal'], data['macdhist'] = ta.MACD(data['Close'].values)
    data['sar'] = ta.SAR(data['High'].values, data['Low'].values)
    features_list +=['upper_band','middle_band','lower_band','macd','sar']
    # Create target
    data['target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)
    
    # Drop null values
    data.dropna(inplace=True)
    
    # Create feature matrix and target vector
    X = data[features_list]
    y = data['target']
    
    # Scale data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Predict
    y_pred = model.predict(X_scaled)
    
    data['predicted'] = y_pred
    
    # Create strategy returns
    data['strategy_returns'] = data['returns'].shift(-1) * data['predicted']
    
    # Return the last cumulative return
    bnh_returns = data['returns'].cumsum()[-1]
    
    # Return the last cumulative strategy return
    # we need to drop the last nan value
    data.dropna(inplace=True)
    strategy_returns = data['strategy_returns'].cumsum()[-1]
    
    plt.figure(figsize=(10, 6))
    plt.plot(data['returns'].cumsum())
    plt.plot(data['strategy_returns'].cumsum())
    plt.xlabel('Time')
    plt.ylabel('Cumulative Returns')
    plt.title('Returns Comparison')
    plt.legend(["Buy and Hold Returns","Strategy Returns"])
    plt.show()
    
    return bnh_returns, strategy_returns, data

In [ ]:
# Read backtest data
backtest_data = pd.read_csv('EURUSD_backtest.csv', index_col=0, parse_dates=True)

# Backtest the strategy
bnh_returns, s_returns, data = backtest(backtest_data, model)

data

In [ ]:
print('Buy and Hold Returns:', bnh_returns)
print('Strategy Returns:', s_returns)